# Train your first Deep Reinforcement Learning Agent

PYTHON 3.9

In [1]:
from fireplace import cards, exceptions, utils
from fireplace.player import Player
from fireplace.game import Game
from fireplace.deck import Deck
from fireplace.utils import get_script_definition, random_draft

In [2]:
from hearthstone.enums import PlayState, Step, Mulligan, State, CardClass, Race, CardSet, CardType

In [3]:
import re
import string
import random

In [8]:
cards.db.initialize()

[fireplace.__init__]: Initializing card database
[fireplace.__init__]: Merged 25713 cards


In [9]:
GREEN = "\033[92m"
RED = "\033[91m"
ENDC = "\033[0m"
PREFIXES = {
    GREEN: "Implemented",
    RED: "Not implemented",
}
implemented_cards = []
unimplemented_cards = []

In [10]:
SOLVED_KEYWORDS = [
    "Windfury", "Charge", "Divine Shield", "Taunt", "Stealth", "Poisonous",
    r"Can't be targeted by spells or Hero Powers\.",
    r"Can't attack\.",
    "Destroy any minion damaged by this minion.",
    r"Your Hero Power deals \d+ extra damage.",
    r"Spell Damage \+\d+",
    r"Overload: \(\d+\)",
]

In [11]:
DUMMY_CARDS = (
    "PlaceholderCard",  # Placeholder Card
    "CS2_022e",  # Polymorph
    "EX1_246e",  # Hexxed
    "EX1_345t",  # Shadow of Nothing
    "GAME_006",  # NOOOOOOOOOOOO
    "LOEA04_27",  # Animated Statue
    "Mekka4e",  # Transformed
    "NEW1_025e",  # Bolstered (Unused)
    "TU4c_005",  # Hidden Gnome
    "TU4c_007",  # Mukla's Big Brother

    # Dynamic buffs set by their parent
    "CS2_236e",  # Divine Spirit
    "EX1_304e",  # Consume (Void Terror)
    "LOE_030e",  # Hollow (Unused)
    "NEW1_018e",  # Treasure Crazed (Bloodsail Raider)
)

In [12]:
for id in sorted(cards.db):
    card = cards.db[id]
    ret = card.description
    ret = re.sub("<i>.+</i>", "", ret)
    ret = re.sub("(<b>|</b>)", "", ret)
    ret = re.sub("(" + "|".join(SOLVED_KEYWORDS) + ")", "", ret)
    ret = re.sub("<[^>]*>", "", ret)
    exclude_chars = string.punctuation + string.whitespace
    ret = "".join([ch for ch in ret if ch not in exclude_chars])
    description = ret
    implemented = False

    if not description:
        # Minions without card text or with basic abilities are implemented
        implemented = True
    elif card.card_set == CardSet.CREDITS:
        implemented = True

    if id in DUMMY_CARDS:
        implemented = True

    carddef = get_script_definition(id)
    if carddef:
        implemented = True

    color = GREEN if implemented else RED
    name = color + "%s: %s" % (PREFIXES[color], card.name) + ENDC

    if implemented:
        implemented_cards.append(card.id)
    else:
        unimplemented_cards.append(card.id)

IMPLEMENTED_CARDS = len(implemented_cards)
UNIMPLEMENTED_CARDS = len(unimplemented_cards)

print("IMPLEMENTED CARDS: "+str(IMPLEMENTED_CARDS))
print("UNIMPLEMENTED CARDS: "+str(UNIMPLEMENTED_CARDS))

IMPLEMENTED CARDS: 7912
UNIMPLEMENTED CARDS: 17801


In [13]:
implemented_cards[0:5]

['AIBot_DemonHunterTrainee_001_hb',
 'AIBot_DruidTrainee_002_hb',
 'AIBot_HunterTrainee_003_hb',
 'AIBot_MageTrainee_004_hb',
 'AIBot_PaladinTrainee_005_hb']

In [14]:
c=cards.db['AIBot_DemonHunterTrainee_001_hb']

In [17]:
c

<AIBot_DemonHunterTrainee_001_hb: 'DemonHunter Trainee'>

In [18]:
c.strings

{<GameTag.CARDNAME: 185>: {'deDE': 'Dämonenjägerlehrling',
  'enUS': 'DemonHunter Trainee',
  'esES': 'Aprendiz de cazador de demonios',
  'esMX': 'Aprendiz de cazador de demonios',
  'frFR': 'Recrue chasseur de démons',
  'itIT': 'Apprendista Cacciatore di Demoni',
  'jaJP': 'デーモンハンター訓練生',
  'koKR': '악마사냥꾼 수련생',
  'plPL': 'Początkujący łowca demonów',
  'ptBR': 'Aprendiz de Caçador de Demônios',
  'ruRU': 'Иллидари-новобранец',
  'thTH': 'ดีมอนฮันเตอร์ฝึกหัด',
  'zhCN': '恶魔猎手训练假人',
  'zhTW': '惡魔獵人新手'},
 <GameTag.CARDTEXT: 184>: {},
 <GameTag.FLAVORTEXT: 351>: {},
 <GameTag.HOW_TO_EARN: 364>: {},
 <GameTag.HOW_TO_EARN_GOLDEN: 365>: {},
 <GameTag.CardTextInPlay: 252>: {},
 <GameTag.TARGETING_ARROW_TEXT: 325>: {},
 <GameTag.ARTISTNAME: 342>: '',
 <GameTag.LocalizationNotes: 344>: ''}

In [10]:
for id in sorted(cards.db):
    card = cards.db[id]
    card.divine_shield 

## Expresiones regulares

In [33]:
for id in sorted(cards.db):
    card = cards.db[id]
    ret = card.description
    ret_org=ret
    ret=ret.replace("\n"," ") 
    ret = re.sub("<i>.+</i>", "", ret,flags=re.IGNORECASE)
    ret = re.sub(r"<\/?b[^>]*>", "", ret)
    ret = re.sub("(" + "|".join(SOLVED_KEYWORDS) + ")", "", ret)
    ret2 = re.sub("<[^>]*>", "", ret)

## Class Autonum

Para qué vale??

In [ ]:
from enum import Enum

class AutoNumber(Enum):
	def __new__(cls):
		value = len(cls.__members__)  # note no + 1
		obj = object.__new__(cls)
		obj._value_ = value
		return obj

class Move(AutoNumber):
	end_turn = ()
	hero_power = ()
	minion_attack = ()
	hero_attack = ()
	play_card = ()
	mulligan = ()
	choice = ()

## Modify hearthstone environment from nested to unnested

In [24]:
from gymnasium import spaces
modify={"myhand0effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand1effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand2effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand3effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand4effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand5effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand6effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand7effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand8effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand9effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myfield0effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myfield1effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myfield2effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myfield3effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myfield4effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myfield5effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myfield6effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "oppfield0effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "oppfield1effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "oppfield2effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "oppfield3effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "oppfield4effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "oppfield5effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "oppfield6effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            )}

In [28]:
modify["myhand0effects"].keys()

KeysView(Dict('aura': Discrete(2), 'cantbetargeted': Discrete(2), 'charge': Discrete(2), 'deathrattle': Discrete(2), 'divineshield': Discrete(2), 'frozen': Discrete(2), 'poisonous': Discrete(2), 'silenced': Discrete(2), 'stealth': Discrete(2), 'taunt': Discrete(2), 'windfury': Discrete(2)))

In [30]:
for k,v in modify.items():
    for kk,vv in v.items():
        print('"'+k+"_"+kk+'": spaces.Discrete(2),')

"myhand0effects_aura": spaces.Discrete(2),
"myhand0effects_cantbetargeted": spaces.Discrete(2),
"myhand0effects_charge": spaces.Discrete(2),
"myhand0effects_deathrattle": spaces.Discrete(2),
"myhand0effects_divineshield": spaces.Discrete(2),
"myhand0effects_frozen": spaces.Discrete(2),
"myhand0effects_poisonous": spaces.Discrete(2),
"myhand0effects_silenced": spaces.Discrete(2),
"myhand0effects_stealth": spaces.Discrete(2),
"myhand0effects_taunt": spaces.Discrete(2),
"myhand0effects_windfury": spaces.Discrete(2),
"myhand1effects_aura": spaces.Discrete(2),
"myhand1effects_cantbetargeted": spaces.Discrete(2),
"myhand1effects_charge": spaces.Discrete(2),
"myhand1effects_deathrattle": spaces.Discrete(2),
"myhand1effects_divineshield": spaces.Discrete(2),
"myhand1effects_frozen": spaces.Discrete(2),
"myhand1effects_poisonous": spaces.Discrete(2),
"myhand1effects_silenced": spaces.Discrete(2),
"myhand1effects_stealth": spaces.Discrete(2),
"myhand1effects_taunt": spaces.Discrete(2),
"myhand1e

Esta parte está cambiada a mano:

In [ ]:
"myhand2effects_windfury", 1 if 2 < len(p1.hand) and p1.hand[2].windfury else 0,
"myhand2effects_divineshield", 1 if 2 < len(p1.hand) and p1.hand[2].divine_shield else 0,
"myhand2effects_charge", 1 if 2 < len(p1.hand) and p1.hand[2].charge else 0,
"myhand2effects_taunt", 1 if 2 < len(p1.hand) and p1.hand[2].taunt else 0,
"myhand2effects_stealth", 1 if 2 < len(p1.hand) and p1.hand[2].stealthed else 0,
"myhand2effects_poisonous", 1 if 2 < len(p1.hand) and p1.hand[2].poisonous else 0,
"myhand2effects_cantbetargeted", 1 if 2 < len(p1.hand) and (p1.hand[2].cant_be_targeted_by_abilities or p1.hand[2].cant_be_targeted_by_hero_powers) else 0,
"myhand2effects_aura", 1 if 2 < len(p1.hand) and p1.hand[2].aura else 0,
"myhand2effects_deathrattle", 1 if 2 < len(p1.hand) and p1.hand[2].has_deathrattle else 0,
"myhand2effects_frozen", 1 if 2 < len(p1.hand) and p1.hand[2].frozen else 0,
"myhand2effects_silenced", 1 if 2 < len(p1.hand) and p1.hand[2].silenced else 0,
"myhand3effects_windfury", 1 if 3 < len(p1.hand) and p1.hand[3].windfury else 0,
"myhand3effects_divineshield", 1 if 3 < len(p1.hand) and p1.hand[3].divine_shield else 0,
"myhand3effects_charge", 1 if 3 < len(p1.hand) and p1.hand[3].charge else 0,
"myhand3effects_taunt", 1 if 3 < len(p1.hand) and p1.hand[3].taunt else 0,
"myhand3effects_stealth", 1 if 3 < len(p1.hand) and p1.hand[3].stealthed else 0,
"myhand3effects_poisonous", 1 if 3 < len(p1.hand) and p1.hand[3].poisonous else 0,
"myhand3effects_cantbetargeted", 1 if 3 < len(p1.hand) and (p1.hand[3].cant_be_targeted_by_abilities or p1.hand[3].cant_be_targeted_by_hero_powers) else 0,
"myhand3effects_aura", 1 if 3 < len(p1.hand) and p1.hand[3].aura else 0,
"myhand3effects_deathrattle", 1 if 3 < len(p1.hand) and p1.hand[3].has_deathrattle else 0,
"myhand3effects_frozen", 1 if 3 < len(p1.hand) and p1.hand[3].frozen else 0,
"myhand3effects_silenced", 1 if 3 < len(p1.hand) and p1.hand[3].silenced else 0,
"myhand4effects_windfury", 1 if 4 < len(p1.hand) and p1.hand[4].windfury else 0,
"myhand4effects_divineshield", 1 if 4 < len(p1.hand) and p1.hand[4].divine_shield else 0,
"myhand4effects_charge", 1 if 4 < len(p1.hand) and p1.hand[4].charge else 0,
"myhand4effects_taunt", 1 if 4 < len(p1.hand) and p1.hand[4].taunt else 0,
"myhand4effects_stealth", 1 if 4 < len(p1.hand) and p1.hand[4].stealthed else 0,
"myhand4effects_poisonous", 1 if 4 < len(p1.hand) and p1.hand[4].poisonous else 0,
"myhand4effects_cantbetargeted", 1 if 4 < len(p1.hand) and (p1.hand[4].cant_be_targeted_by_abilities or p1.hand[4].cant_be_targeted_by_hero_powers) else 0,
"myhand4effects_aura", 1 if 4 < len(p1.hand) and p1.hand[4].aura else 0,
"myhand4effects_deathrattle", 1 if 4 < len(p1.hand) and p1.hand[4].has_deathrattle else 0,
"myhand4effects_frozen", 1 if 4 < len(p1.hand) and p1.hand[4].frozen else 0,
"myhand4effects_silenced", 1 if 4 < len(p1.hand) and p1.hand[4].silenced else 0,
"myhand5effects_windfury", 1 if 5 < len(p1.hand) and p1.hand[5].windfury else 0,
"myhand5effects_divineshield", 1 if 5 < len(p1.hand) and p1.hand[5].divine_shield else 0,
"myhand5effects_charge", 1 if 5 < len(p1.hand) and p1.hand[5].charge else 0,
"myhand5effects_taunt", 1 if 5 < len(p1.hand) and p1.hand[5].taunt else 0,
"myhand5effects_stealth", 1 if 5 < len(p1.hand) and p1.hand[5].stealthed else 0,
"myhand5effects_poisonous", 1 if 5 < len(p1.hand) and p1.hand[5].poisonous else 0,
"myhand5effects_cantbetargeted", 1 if 5 < len(p1.hand) and (p1.hand[5].cant_be_targeted_by_abilities or p1.hand[5].cant_be_targeted_by_hero_powers) else 0,
"myhand5effects_aura", 1 if 5 < len(p1.hand) and p1.hand[5].aura else 0,
"myhand5effects_deathrattle", 1 if 5 < len(p1.hand) and p1.hand[5].has_deathrattle else 0,
"myhand5effects_frozen", 1 if 5 < len(p1.hand) and p1.hand[5].frozen else 0,
"myhand5effects_silenced", 1 if 5 < len(p1.hand) and p1.hand[5].silenced else 0,
"myhand6effects_windfury", 1 if 6 < len(p1.hand) and p1.hand[6].windfury else 0,
"myhand6effects_divineshield", 1 if 6 < len(p1.hand) and p1.hand[6].divine_shield else 0,
"myhand6effects_charge", 1 if 6 < len(p1.hand) and p1.hand[6].charge else 0,
"myhand6effects_taunt", 1 if 6 < len(p1.hand) and p1.hand[6].taunt else 0,
"myhand6effects_stealth", 1 if 6 < len(p1.hand) and p1.hand[6].stealthed else 0,
"myhand6effects_poisonous", 1 if 6 < len(p1.hand) and p1.hand[6].poisonous else 0,
"myhand6effects_cantbetargeted", 1 if 6 < len(p1.hand) and (p1.hand[6].cant_be_targeted_by_abilities or p1.hand[6].cant_be_targeted_by_hero_powers) else 0,
"myhand6effects_aura", 1 if 6 < len(p1.hand) and p1.hand[6].aura else 0,
"myhand6effects_deathrattle", 1 if 6 < len(p1.hand) and p1.hand[6].has_deathrattle else 0,
"myhand6effects_frozen", 1 if 6 < len(p1.hand) and p1.hand[6].frozen else 0,
"myhand6effects_silenced", 1 if 6 < len(p1.hand) and p1.hand[6].silenced else 0,
"myhand7effects_windfury", 1 if 7 < len(p1.hand) and p1.hand[7].windfury else 0,
"myhand7effects_divineshield", 1 if 7 < len(p1.hand) and p1.hand[7].divine_shield else 0,
"myhand7effects_charge", 1 if 7 < len(p1.hand) and p1.hand[7].charge else 0,
"myhand7effects_taunt", 1 if 7 < len(p1.hand) and p1.hand[7].taunt else 0,
"myhand7effects_stealth", 1 if 7 < len(p1.hand) and p1.hand[7].stealthed else 0,
"myhand7effects_poisonous", 1 if 7 < len(p1.hand) and p1.hand[7].poisonous else 0,
"myhand7effects_cantbetargeted", 1 if 7 < len(p1.hand) and (p1.hand[7].cant_be_targeted_by_abilities or p1.hand[7].cant_be_targeted_by_hero_powers) else 0,
"myhand7effects_aura", 1 if 7 < len(p1.hand) and p1.hand[7].aura else 0,
"myhand7effects_deathrattle", 1 if 7 < len(p1.hand) and p1.hand[7].has_deathrattle else 0,
"myhand7effects_frozen", 1 if 7 < len(p1.hand) and p1.hand[7].frozen else 0,
"myhand7effects_silenced", 1 if 7 < len(p1.hand) and p1.hand[7].silenced else 0,
"myhand8effects_windfury", 1 if 8 < len(p1.hand) and p1.hand[8].windfury else 0,
"myhand8effects_divineshield", 1 if 8 < len(p1.hand) and p1.hand[8].divine_shield else 0,
"myhand8effects_charge", 1 if 8 < len(p1.hand) and p1.hand[8].charge else 0,
"myhand8effects_taunt", 1 if 8 < len(p1.hand) and p1.hand[8].taunt else 0,
"myhand8effects_stealth", 1 if 8 < len(p1.hand) and p1.hand[8].stealthed else 0,
"myhand8effects_poisonous", 1 if 8 < len(p1.hand) and p1.hand[8].poisonous else 0,
"myhand8effects_cantbetargeted", 1 if 8 < len(p1.hand) and (p1.hand[8].cant_be_targeted_by_abilities or p1.hand[8].cant_be_targeted_by_hero_powers) else 0,
"myhand8effects_aura", 1 if 8 < len(p1.hand) and p1.hand[8].aura else 0,
"myhand8effects_deathrattle", 1 if 8 < len(p1.hand) and p1.hand[8].has_deathrattle else 0,
"myhand8effects_frozen", 1 if 8 < len(p1.hand) and p1.hand[8].frozen else 0,
"myhand8effects_silenced", 1 if 8 < len(p1.hand) and p1.hand[8].silenced else 0,
"myhand9effects_windfury", 1 if 9 < len(p1.hand) and p1.hand[9].windfury else 0,
"myhand9effects_divineshield", 1 if 9 < len(p1.hand) and p1.hand[9].divine_shield else 0,
"myhand9effects_charge", 1 if 9 < len(p1.hand) and p1.hand[9].charge else 0,
"myhand9effects_taunt", 1 if 9 < len(p1.hand) and p1.hand[9].taunt else 0,
"myhand9effects_stealth", 1 if 9 < len(p1.hand) and p1.hand[9].stealthed else 0,
"myhand9effects_poisonous", 1 if 9 < len(p1.hand) and p1.hand[9].poisonous else 0,
"myhand9effects_cantbetargeted", 1 if 9 < len(p1.hand) and (p1.hand[9].cant_be_targeted_by_abilities or p1.hand[9].cant_be_targeted_by_hero_powers) else 0,
"myhand9effects_aura", 1 if 9 < len(p1.hand) and p1.hand[9].aura else 0,
"myhand9effects_deathrattle", 1 if 9 < len(p1.hand) and p1.hand[9].has_deathrattle else 0,
"myhand9effects_frozen", 1 if 9 < len(p1.hand) and p1.hand[9].frozen else 0,
"myhand9effects_silenced", 1 if 9 < len(p1.hand) and p1.hand[9].silenced else 0,
"myfield0effects_windfury", 1 if 0 < len(p1.field) and p1.field[0].windfury else 0,
"myfield0effects_divineshield", 1 if 0 < len(p1.field) and p1.field[0].divine_shield else 0,
"myfield0effects_charge", 1 if 0 < len(p1.field) and p1.field[0].charge else 0,
"myfield0effects_taunt", 1 if 0 < len(p1.field) and p1.field[0].taunt else 0,
"myfield0effects_stealth", 1 if 0 < len(p1.field) and p1.field[0].stealthed else 0,
"myfield0effects_poisonous", 1 if 0 < len(p1.field) and p1.field[0].poisonous else 0,
"myfield0effects_cantbetargeted", 1 if 0 < len(p1.field) and (p1.field[0].cant_be_targeted_by_abilities or p1.field[0].cant_be_targeted_by_hero_powers) else 0,
"myfield0effects_aura", 1 if 0 < len(p1.field) and p1.field[0].aura else 0,
"myfield0effects_deathrattle", 1 if 0 < len(p1.field) and p1.field[0].has_deathrattle else 0,
"myfield0effects_frozen", 1 if 0 < len(p1.field) and p1.field[0].frozen else 0,
"myfield0effects_silenced", 1 if 0 < len(p1.field) and p1.field[0].silenced else 0,
"myfield1effects_windfury", 1 if 1 < len(p1.field) and p1.field[1].windfury else 0,
"myfield1effects_divineshield", 1 if 1 < len(p1.field) and p1.field[1].divine_shield else 0,
"myfield1effects_charge", 1 if 1 < len(p1.field) and p1.field[1].charge else 0,
"myfield1effects_taunt", 1 if 1 < len(p1.field) and p1.field[1].taunt else 0,
"myfield1effects_stealth", 1 if 1 < len(p1.field) and p1.field[1].stealthed else 0,
"myfield1effects_poisonous", 1 if 1 < len(p1.field) and p1.field[1].poisonous else 0,
"myfield1effects_cantbetargeted", 1 if 1 < len(p1.field) and (p1.field[1].cant_be_targeted_by_abilities or p1.field[1].cant_be_targeted_by_hero_powers) else 0,
"myfield1effects_aura", 1 if 1 < len(p1.field) and p1.field[1].aura else 0,
"myfield1effects_deathrattle", 1 if 1 < len(p1.field) and p1.field[1].has_deathrattle else 0,
"myfield1effects_frozen", 1 if 1 < len(p1.field) and p1.field[1].frozen else 0,
"myfield1effects_silenced", 1 if 1 < len(p1.field) and p1.field[1].silenced else 0,
"myfield2effects_windfury", 1 if 2 < len(p1.field) and p1.field[2].windfury else 0,
"myfield2effects_divineshield", 1 if 2 < len(p1.field) and p1.field[2].divine_shield else 0,
"myfield2effects_charge", 1 if 2 < len(p1.field) and p1.field[2].charge else 0,
"myfield2effects_taunt", 1 if 2 < len(p1.field) and p1.field[2].taunt else 0,
"myfield2effects_stealth", 1 if 2 < len(p1.field) and p1.field[2].stealthed else 0,
"myfield2effects_poisonous", 1 if 2 < len(p1.field) and p1.field[2].poisonous else 0,
"myfield2effects_cantbetargeted", 1 if 2 < len(p1.field) and (p1.field[2].cant_be_targeted_by_abilities or p1.field[2].cant_be_targeted_by_hero_powers) else 0,
"myfield2effects_aura", 1 if 2 < len(p1.field) and p1.field[2].aura else 0,
"myfield2effects_deathrattle", 1 if 2 < len(p1.field) and p1.field[2].has_deathrattle else 0,
"myfield2effects_frozen", 1 if 2 < len(p1.field) and p1.field[2].frozen else 0,
"myfield2effects_silenced", 1 if 2 < len(p1.field) and p1.field[2].silenced else 0,
"myfield3effects_windfury", 1 if 3 < len(p1.field) and p1.field[3].windfury else 0,
"myfield3effects_divineshield", 1 if 3 < len(p1.field) and p1.field[3].divine_shield else 0,
"myfield3effects_charge", 1 if 3 < len(p1.field) and p1.field[3].charge else 0,
"myfield3effects_taunt", 1 if 3 < len(p1.field) and p1.field[3].taunt else 0,
"myfield3effects_stealth", 1 if 3 < len(p1.field) and p1.field[3].stealthed else 0,
"myfield3effects_poisonous", 1 if 3 < len(p1.field) and p1.field[3].poisonous else 0,
"myfield3effects_cantbetargeted", 1 if 3 < len(p1.field) and (p1.field[3].cant_be_targeted_by_abilities or p1.field[3].cant_be_targeted_by_hero_powers) else 0,
"myfield3effects_aura", 1 if 3 < len(p1.field) and p1.field[3].aura else 0,
"myfield3effects_deathrattle", 1 if 3 < len(p1.field) and p1.field[3].has_deathrattle else 0,
"myfield3effects_frozen", 1 if 3 < len(p1.field) and p1.field[3].frozen else 0,
"myfield3effects_silenced", 1 if 3 < len(p1.field) and p1.field[3].silenced else 0,
"myfield4effects_windfury", 1 if 4 < len(p1.field) and p1.field[4].windfury else 0,
"myfield4effects_divineshield", 1 if 4 < len(p1.field) and p1.field[4].divine_shield else 0,
"myfield4effects_charge", 1 if 4 < len(p1.field) and p1.field[4].charge else 0,
"myfield4effects_taunt", 1 if 4 < len(p1.field) and p1.field[4].taunt else 0,
"myfield4effects_stealth", 1 if 4 < len(p1.field) and p1.field[4].stealthed else 0,
"myfield4effects_poisonous", 1 if 4 < len(p1.field) and p1.field[4].poisonous else 0,
"myfield4effects_cantbetargeted", 1 if 4 < len(p1.field) and (p1.field[4].cant_be_targeted_by_abilities or p1.field[4].cant_be_targeted_by_hero_powers) else 0,
"myfield4effects_aura", 1 if 4 < len(p1.field) and p1.field[4].aura else 0,
"myfield4effects_deathrattle", 1 if 4 < len(p1.field) and p1.field[4].has_deathrattle else 0,
"myfield4effects_frozen", 1 if 4 < len(p1.field) and p1.field[4].frozen else 0,
"myfield4effects_silenced", 1 if 4 < len(p1.field) and p1.field[4].silenced else 0,
"myfield5effects_windfury", 1 if 5 < len(p1.field) and p1.field[5].windfury else 0,
"myfield5effects_divineshield", 1 if 5 < len(p1.field) and p1.field[5].divine_shield else 0,
"myfield5effects_charge", 1 if 5 < len(p1.field) and p1.field[5].charge else 0,
"myfield5effects_taunt", 1 if 5 < len(p1.field) and p1.field[5].taunt else 0,
"myfield5effects_stealth", 1 if 5 < len(p1.field) and p1.field[5].stealthed else 0,
"myfield5effects_poisonous", 1 if 5 < len(p1.field) and p1.field[5].poisonous else 0,
"myfield5effects_cantbetargeted", 1 if 5 < len(p1.field) and (p1.field[5].cant_be_targeted_by_abilities or p1.field[5].cant_be_targeted_by_hero_powers) else 0,
"myfield5effects_aura", 1 if 5 < len(p1.field) and p1.field[5].aura else 0,
"myfield5effects_deathrattle", 1 if 5 < len(p1.field) and p1.field[5].has_deathrattle else 0,
"myfield5effects_frozen", 1 if 5 < len(p1.field) and p1.field[5].frozen else 0,
"myfield5effects_silenced", 1 if 5 < len(p1.field) and p1.field[5].silenced else 0,
"myfield6effects_windfury", 1 if 6 < len(p1.field) and p1.field[6].windfury else 0,
"myfield6effects_divineshield", 1 if 6 < len(p1.field) and p1.field[6].divine_shield else 0,
"myfield6effects_charge", 1 if 6 < len(p1.field) and p1.field[6].charge else 0,
"myfield6effects_taunt", 1 if 6 < len(p1.field) and p1.field[6].taunt else 0,
"myfield6effects_stealth", 1 if 6 < len(p1.field) and p1.field[6].stealthed else 0,
"myfield6effects_poisonous", 1 if 6 < len(p1.field) and p1.field[6].poisonous else 0,
"myfield6effects_cantbetargeted", 1 if 6 < len(p1.field) and (p1.field[6].cant_be_targeted_by_abilities or p1.field[6].cant_be_targeted_by_hero_powers) else 0,
"myfield6effects_aura", 1 if 6 < len(p1.field) and p1.field[6].aura else 0,
"myfield6effects_deathrattle", 1 if 6 < len(p1.field) and p1.field[6].has_deathrattle else 0,
"myfield6effects_frozen", 1 if 6 < len(p1.field) and p1.field[6].frozen else 0,
"myfield6effects_silenced", 1 if 6 < len(p1.field) and p1.field[6].silenced else 0,
"oppfield0effects_windfury", 1 if 0 < len(p2.field) and p2.field[0].windfury else 0,
"oppfield0effects_divineshield", 1 if 0 < len(p2.field) and p2.field[0].divine_shield else 0,
"oppfield0effects_charge", 1 if 0 < len(p2.field) and p2.field[0].charge else 0,
"oppfield0effects_taunt", 1 if 0 < len(p2.field) and p2.field[0].taunt else 0,
"oppfield0effects_stealth", 1 if 0 < len(p2.field) and p2.field[0].stealthed else 0,
"oppfield0effects_poisonous", 1 if 0 < len(p2.field) and p2.field[0].poisonous else 0,
"oppfield0effects_cantbetargeted", 1 if 0 < len(p2.field) and (p2.field[0].cant_be_targeted_by_abilities or p2.field[0].cant_be_targeted_by_hero_powers) else 0,
"oppfield0effects_aura", 1 if 0 < len(p2.field) and p2.field[0].aura else 0,
"oppfield0effects_deathrattle", 1 if 0 < len(p2.field) and p2.field[0].has_deathrattle else 0,
"oppfield0effects_frozen", 1 if 0 < len(p2.field) and p2.field[0].frozen else 0,
"oppfield0effects_silenced", 1 if 0 < len(p2.field) and p2.field[0].silenced else 0,
"oppfield1effects_windfury", 1 if 1 < len(p2.field) and p2.field[1].windfury else 0,
"oppfield1effects_divineshield", 1 if 1 < len(p2.field) and p2.field[1].divine_shield else 0,
"oppfield1effects_charge", 1 if 1 < len(p2.field) and p2.field[1].charge else 0,
"oppfield1effects_taunt", 1 if 1 < len(p2.field) and p2.field[1].taunt else 0,
"oppfield1effects_stealth", 1 if 1 < len(p2.field) and p2.field[1].stealthed else 0,
"oppfield1effects_poisonous", 1 if 1 < len(p2.field) and p2.field[1].poisonous else 0,
"oppfield1effects_cantbetargeted", 1 if 1 < len(p2.field) and (p2.field[1].cant_be_targeted_by_abilities or p2.field[1].cant_be_targeted_by_hero_powers) else 0,
"oppfield1effects_aura", 1 if 1 < len(p2.field) and p2.field[1].aura else 0,
"oppfield1effects_deathrattle", 1 if 1 < len(p2.field) and p2.field[1].has_deathrattle else 0,
"oppfield1effects_frozen", 1 if 1 < len(p2.field) and p2.field[1].frozen else 0,
"oppfield1effects_silenced", 1 if 1 < len(p2.field) and p2.field[1].silenced else 0,
"oppfield2effects_windfury", 1 if 2 < len(p2.field) and p2.field[2].windfury else 0,
"oppfield2effects_divineshield", 1 if 2 < len(p2.field) and p2.field[2].divine_shield else 0,
"oppfield2effects_charge", 1 if 2 < len(p2.field) and p2.field[2].charge else 0,
"oppfield2effects_taunt", 1 if 2 < len(p2.field) and p2.field[2].taunt else 0,
"oppfield2effects_stealth", 1 if 2 < len(p2.field) and p2.field[2].stealthed else 0,
"oppfield2effects_poisonous", 1 if 2 < len(p2.field) and p2.field[2].poisonous else 0,
"oppfield2effects_cantbetargeted", 1 if 2 < len(p2.field) and (p2.field[2].cant_be_targeted_by_abilities or p2.field[2].cant_be_targeted_by_hero_powers) else 0,
"oppfield2effects_aura", 1 if 2 < len(p2.field) and p2.field[2].aura else 0,
"oppfield2effects_deathrattle", 1 if 2 < len(p2.field) and p2.field[2].has_deathrattle else 0,
"oppfield2effects_frozen", 1 if 2 < len(p2.field) and p2.field[2].frozen else 0,
"oppfield2effects_silenced", 1 if 2 < len(p2.field) and p2.field[2].silenced else 0,
"oppfield3effects_windfury", 1 if 3 < len(p2.field) and p2.field[3].windfury else 0,
"oppfield3effects_divineshield", 1 if 3 < len(p2.field) and p2.field[3].divine_shield else 0,
"oppfield3effects_charge", 1 if 3 < len(p2.field) and p2.field[3].charge else 0,
"oppfield3effects_taunt", 1 if 3 < len(p2.field) and p2.field[3].taunt else 0,
"oppfield3effects_stealth", 1 if 3 < len(p2.field) and p2.field[3].stealthed else 0,
"oppfield3effects_poisonous", 1 if 3 < len(p2.field) and p2.field[3].poisonous else 0,
"oppfield3effects_cantbetargeted", 1 if 3 < len(p2.field) and (p2.field[3].cant_be_targeted_by_abilities or p2.field[3].cant_be_targeted_by_hero_powers) else 0,
"oppfield3effects_aura", 1 if 3 < len(p2.field) and p2.field[3].aura else 0,
"oppfield3effects_deathrattle", 1 if 3 < len(p2.field) and p2.field[3].has_deathrattle else 0,
"oppfield3effects_frozen", 1 if 3 < len(p2.field) and p2.field[3].frozen else 0,
"oppfield3effects_silenced", 1 if 3 < len(p2.field) and p2.field[3].silenced else 0,
"oppfield4effects_windfury", 1 if 4 < len(p2.field) and p2.field[4].windfury else 0,
"oppfield4effects_divineshield", 1 if 4 < len(p2.field) and p2.field[4].divine_shield else 0,
"oppfield4effects_charge", 1 if 4 < len(p2.field) and p2.field[4].charge else 0,
"oppfield4effects_taunt", 1 if 4 < len(p2.field) and p2.field[4].taunt else 0,
"oppfield4effects_stealth", 1 if 4 < len(p2.field) and p2.field[4].stealthed else 0,
"oppfield4effects_poisonous", 1 if 4 < len(p2.field) and p2.field[4].poisonous else 0,
"oppfield4effects_cantbetargeted", 1 if 4 < len(p2.field) and (p2.field[4].cant_be_targeted_by_abilities or p2.field[4].cant_be_targeted_by_hero_powers) else 0,
"oppfield4effects_aura", 1 if 4 < len(p2.field) and p2.field[4].aura else 0,
"oppfield4effects_deathrattle", 1 if 4 < len(p2.field) and p2.field[4].has_deathrattle else 0,
"oppfield4effects_frozen", 1 if 4 < len(p2.field) and p2.field[4].frozen else 0,
"oppfield4effects_silenced", 1 if 4 < len(p2.field) and p2.field[4].silenced else 0,
"oppfield5effects_windfury", 1 if 5 < len(p2.field) and p2.field[5].windfury else 0,
"oppfield5effects_divineshield", 1 if 5 < len(p2.field) and p2.field[5].divine_shield else 0,
"oppfield5effects_charge", 1 if 5 < len(p2.field) and p2.field[5].charge else 0,
"oppfield5effects_taunt", 1 if 5 < len(p2.field) and p2.field[5].taunt else 0,
"oppfield5effects_stealth", 1 if 5 < len(p2.field) and p2.field[5].stealthed else 0,
"oppfield5effects_poisonous", 1 if 5 < len(p2.field) and p2.field[5].poisonous else 0,
"oppfield5effects_cantbetargeted", 1 if 5 < len(p2.field) and (p2.field[5].cant_be_targeted_by_abilities or p2.field[5].cant_be_targeted_by_hero_powers) else 0,
"oppfield5effects_aura", 1 if 5 < len(p2.field) and p2.field[5].aura else 0,
"oppfield5effects_deathrattle", 1 if 5 < len(p2.field) and p2.field[5].has_deathrattle else 0,
"oppfield5effects_frozen", 1 if 5 < len(p2.field) and p2.field[5].frozen else 0,
"oppfield5effects_silenced", 1 if 5 < len(p2.field) and p2.field[5].silenced else 0,
"oppfield6effects_windfury", 1 if 6 < len(p2.field) and p2.field[6].windfury else 0,
"oppfield6effects_divineshield", 1 if 6 < len(p2.field) and p2.field[6].divine_shield else 0,
"oppfield6effects_charge", 1 if 6 < len(p2.field) and p2.field[6].charge else 0,
"oppfield6effects_taunt", 1 if 6 < len(p2.field) and p2.field[6].taunt else 0,
"oppfield6effects_stealth", 1 if 6 < len(p2.field) and p2.field[6].stealthed else 0,
"oppfield6effects_poisonous", 1 if 6 < len(p2.field) and p2.field[6].poisonous else 0,
"oppfield6effects_cantbetargeted", 1 if 6 < len(p2.field) and (p2.field[6].cant_be_targeted_by_abilities or p2.field[6].cant_be_targeted_by_hero_powers) else 0,
"oppfield6effects_aura", 1 if 6 < len(p2.field) and p2.field[6].aura else 0,
"oppfield6effects_deathrattle", 1 if 6 < len(p2.field) and p2.field[6].has_deathrattle else 0,
"oppfield6effects_frozen", 1 if 6 < len(p2.field) and p2.field[6].frozen else 0,
"oppfield6effects_silenced", 1 if 6 < len(p2.field) and p2.field[6].silenced else 0

In [36]:
from hearthstone.enums import PlayState, Step, Mulligan, State, CardClass, Race, CardSet, CardType

list(CardClass)

[<CardClass.INVALID: 0>,
 <CardClass.DEATHKNIGHT: 1>,
 <CardClass.DRUID: 2>,
 <CardClass.HUNTER: 3>,
 <CardClass.MAGE: 4>,
 <CardClass.PALADIN: 5>,
 <CardClass.PRIEST: 6>,
 <CardClass.ROGUE: 7>,
 <CardClass.SHAMAN: 8>,
 <CardClass.WARLOCK: 9>,
 <CardClass.WARRIOR: 10>,
 <CardClass.DREAM: 11>,
 <CardClass.NEUTRAL: 12>,
 <CardClass.WHIZBANG: 13>,
 <CardClass.DEMONHUNTER: 14>]

In [68]:
class HearthstoneUnnestedEnv():
    
    def setup_game(self):
            while True:
                print("iter")
                try:
                    # At the root pretend the player just moved is p2 - p1 has the first move
                    self.playerJustMoved=2
                    self.playerToMove=1
                    self.players_ordered=None
                    self.hero1=random.choice(list(CardClass))
                    self.deck1=None
                    self.hero2=random.choice(list(CardClass))
                    self.deck2=None
                    self.game=None
                    self.lastMovePlayed=None
                    if self.hero1 is None or self.hero2 is None or self.deck1 is None or self.deck2 is None:
                        self.deck1=[]
                        self.deck2=[]
                        self.collection1 = []
                        self.collection2 = []
                        for card in cards.db.keys():
                            if str(card) not in implemented_cards:
                                continue
                            cls = cards.db[card]
                            if not cls.collectible:
                                continue
                            if cls.type == CardType.HERO:
                                # Heroes are collectible...
                                continue
                            if cls.card_class and cls.card_class in [self.hero1, CardClass.NEUTRAL]:
                                # Play with more possibilities
                                self.collection1.append(cls)
                            if cls.card_class and cls.card_class in [self.hero2, CardClass.NEUTRAL]:
                                # Play with more possibilities
                                self.collection2.append(cls)
                            

                        while len(self.deck1) < Deck.MAX_CARDS:
                            card = random.choice(self.collection1)
                            if self.deck1.count(card.id) < card.max_count_in_deck:
                                self.deck1.append(card.id)

                        while len(self.deck2) < Deck.MAX_CARDS:
                            card = random.choice(self.collection2)
                            if self.deck2.count(card.id) < card.max_count_in_deck:
                                self.deck2.append(card.id)

                        self.player1=Player("Player1", self.deck1, self.hero1.default_hero)
                        self.player2=Player("Player2", self.deck2, self.hero2.default_hero)

                        self.game=Game(players=(self.player1, self.player2))
                        self.game.start()
                        self.players_ordered=[self.game.player1, self.game.player2]
                        # At the root pretend the player just moved is p2 - p1 has the first move
                        self.playerJustMoved=2
                        self.playerToMove=1
                        self.lastMovePlayed=None
                        return
                except Exception as ex:
                    print("exception making decks--trying again"+str(ex))

In [69]:
a=HearthstoneUnnestedEnv()
a.setup_game()

iter


[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player1 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Steady Shot')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Steady Shot')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Rexxar')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<Hero ('Rexxar')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Rexxar')>) shuffles their deck
[fireplace.card]: <Minion ('Cauldron Elemental')> moves from <Zone.D

In [70]:
print(len(a.collection1))
print(len(a.collection2))

872
859


In [77]:
a.player1.choice.cards

[<Minion ('Cauldron Elemental')>,
 <Spell ('Cybertech Chip')>,
 <Minion ("Y'Shaarj, Rage Unbound")>]

In [78]:
a.player1.hand

[<Minion ('Cauldron Elemental')>,
 <Spell ('Cybertech Chip')>,
 <Minion ("Y'Shaarj, Rage Unbound")>]

In [79]:
a.game.current_player

**\_\_init\_\_**  Inicializa los espacios de acción y observaciones y llama a **setup_game()**

**reset**  Llama a **setup_game()** y devuelve el estado inicial: **self._get_state()** definida en **gym.Env**

**setup_game()** Realiza varias acciones:
* Elige un HERO para cada jugador
* Crea un deck para cada jugador
* Inicializa el juego e invoca a la funcion START()

**step(action)** El método se limita a invocar a las funciones **_take_action(action)**, **._get_reward()** y **_get_state()**. Se le pasa una de las 869 acciones definidas y devuelve 4 elementos:
* una observación del entorno
* la reward conseguida con esa acción
* buleano diciendo si el episodio a terminado
* info para debugging  

**\_\_getMoves**: Pedirle a Jesús que lo revise. En función del momento del juego, los movimientos posibles son Move.mulligan, Move.choice o Move.play_card, Move.hero_power, Move.minion_attack, Move.hero_attack y Move.end_turn. Ojo, el movimiento Move.end_turn parece que no se añade siempre. Y cuando se meter por el ELSE identificado como "# Play card", valid_moves no sería una lista vacía?? Por qué hace un range(len(valid_moves))???


**\_\_doMove**: Pedirle a Jesús que lo revise. Entre otras cosas, cambia el valor del turno del jugador. 


**_take_action** Empieza calculando cuales son las posibles acciones. Y de todas las posibles acciones eliminar acciones que ya hayan sido seleccionadas antes (y que estarán en la lista **alreadySelectedActions**). El caso es que hay un punto de fallo en el que se eliminan las acciones que ya se han elegido y puede darse la circunstancia que las posibles acciones queden a 0. Por lo que el programa falla. Preguntar a Jesús si tiene sentido.
Quiero entender que un jugador puede hacer todos los movimientos que quiera antes de pasar el turno?? La función **\_\_getMoves** debería de devolver una lista con acciones entre las que se incluya el valor Move.end_turn??

El funcionamiento en general es que la AI que estamos entrenado realiza la acción pedida y la añade a la lista **alreadySelectedActions** para evitar que se vuelva a usar otra vez. En caso de que la acción seleccionada fuera una pérdida de turno para la AI, entonces:
1. se le pasa el turno al otro player (llamado RANDOM) invocando a la función \_\_doMove con la acción
2. se vacía la lista alreadySelectedActions 
3. se obtienen las posibles acciones del RANDOM player
4. se elige una acción random de entre las disponibles
5. se añade a la lista alreadySelectedActions
6. se realiza la acción elegida
7. Se vuelven a calcular las posibles acciones
8. se eliminan las que ya se hayan elegido antes
9. se vuelve a elegir otra random. 
10. Y así hasta que se sevuelva la acción Move.end_turn en cuyo caso se ejecuta, se vacía la lista alreadySelectedActions y se le pasa el turno al jugador AI


Tampoco entiendo porqué si hay 869 acciones que parece que están asignadas, al final, se llena la lista de 869 acciones con repeticiones de las acciones posibles. No debería corresponderse siempre un número con la misma acción????

**\_get_reward**  